<a href="https://colab.research.google.com/github/aditivashivale11/nlp-sessions/blob/main/Hybrid_ML_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import pandas as pd

In [52]:
data = pd.read_csv('/content/apple_stock_data.csv')
print(data.head())

                        Date   Adj Close       Close        High         Low  \
0  2023-11-02 00:00:00+00:00  176.665985  177.570007  177.779999  175.460007   
1  2023-11-03 00:00:00+00:00  175.750671  176.649994  176.820007  173.350006   
2  2023-11-06 00:00:00+00:00  178.317520  179.229996  179.429993  176.210007   
3  2023-11-07 00:00:00+00:00  180.894333  181.820007  182.440002  178.970001   
4  2023-11-08 00:00:00+00:00  181.958893  182.889999  183.449997  181.589996   

         Open    Volume  
0  175.520004  77334800  
1  174.240005  79763700  
2  176.380005  63841300  
3  179.179993  70530000  
4  182.350006  49340300  


In [53]:
print(data['Date'].head())

0    2023-11-02 00:00:00+00:00
1    2023-11-03 00:00:00+00:00
2    2023-11-06 00:00:00+00:00
3    2023-11-07 00:00:00+00:00
4    2023-11-08 00:00:00+00:00
Name: Date, dtype: object


In [54]:
# Convert 'Date' column to datetime and set it as index
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data = data.dropna(subset=['Date'])
data.set_index('Date', inplace=True)

In [55]:
# Ensure 'Close' column exists
if 'Close' not in data.columns:
    raise KeyError("The 'Close' column is missing in the dataset. Available columns are: " + ", ".join(data.columns))

In [56]:
import numpy as np

In [57]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data['Close'] = scaler.fit_transform(data[['Close']])

In [58]:
import numpy as np
def create_sequences(data, seq_length=60):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 60
X, y = create_sequences(data['Close'].values, seq_length)

In [59]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [61]:
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - loss: 0.2704
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0560
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0440
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0169
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0190
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0154
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.0114
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0139
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0098
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0098
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.0103
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.0090
Epoch 13/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 0.0095
Epoch 14/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.0094
Epoch 15/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 0.0079
Epoch 16/20
5/5 ━━━━━━━━━━━━━━━━━━

In [62]:
data['Lag_1'] = data['Close'].shift(1)
data['Lag_2'] = data['Close'].shift(2)
data['Lag_3'] = data['Close'].shift(3)
data = data.dropna()

In [63]:
X_lin = data[['Lag_1', 'Lag_2', 'Lag_3']]
y_lin = data['Close']
X_train_lin, X_test_lin = X_lin[:train_size], X_lin[train_size:]
y_train_lin, y_test_lin = y_lin[:train_size], y_lin[train_size:]

In [64]:
from sklearn.linear_model import LinearRegression
lin_model = LinearRegression()
lin_model.fit(X_train_lin, y_train_lin)

LinearRegression()

In [65]:
X_test_lstm = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
lstm_predictions = lstm_model.predict(X_test_lstm)
lstm_predictions = scaler.inverse_transform(lstm_predictions)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 276ms/step


In [66]:
lin_predictions = lin_model.predict(X_test_lin)
lin_predictions = scaler.inverse_transform(lin_predictions.reshape(-1, 1))

In [68]:
# Assuming lstm_predictions has fewer rows, pad it with zeros to match lin_predictions
padding_size = lin_predictions.shape[0] - lstm_predictions.shape[0]
padding = np.zeros((padding_size, 1))
lstm_predictions_padded = np.concatenate([lstm_predictions, padding])

# Now you can perform the hybrid prediction calculation
hybrid_predictions = (0.7 * lstm_predictions_padded) + (0.3 * lin_predictions)

**Predicting Using Hybrid Model**

In [72]:

lstm_future_pred = []
last_seq = X[-1].reshape(1, seq_length, 1)
for _ in range(10):
  lstm_pred = lstm_model.predict(last_seq)[0, 0]
  lstm_future_pred.append(lstm_pred)
  lstm_pred_reshape = np.array([[lstm_pred]]).reshape(1, 1, 1)
  last_seq = np.append(last_seq[:, 1:, :], lstm_pred_reshape, axis=1)
lstm_future_pred = scaler.inverse_transform(np.array(lstm_future_pred).reshape(-1, 1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [73]:
recent_data = data['Close'].values[-3:]
lin_future_pred = []
for _ in range(10):
  lin_pred = lin_model.predict(recent_data.reshape(1, -1))[0]
  lin_future_pred.append(lin_pred)
  recent_data = np.append(recent_data[1:], lin_pred)
lin_future_pred = scaler.inverse_transform(np.array(lin_future_pred).reshape(-1, 1))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/

In [75]:
hybrid_future_predictions = (0.7 * lstm_future_pred) + (0.3 * lin_future_pred)

In [76]:
future_dates = pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=10)
pred_df = pd.DataFrame({
    'Date' : future_dates,
    'LSTM Preds' : lstm_future_pred.flatten(),
    'Linear Regression Pred' : lin_future_pred.flatten(),
    'Hybrid Model Pred' : hybrid_future_predictions.flatten()
})
print(pred_df)

                       Date  LSTM Preds  Linear Regression Pred  \
0 2024-11-02 00:00:00+00:00  231.984619              230.355192   
1 2024-11-03 00:00:00+00:00  231.692764              225.707291   
2 2024-11-04 00:00:00+00:00  231.451782              222.703426   
3 2024-11-05 00:00:00+00:00  231.249603              230.631535   
4 2024-11-06 00:00:00+00:00  231.076233              225.486380   
5 2024-11-07 00:00:00+00:00  230.923737              222.494588   
6 2024-11-08 00:00:00+00:00  230.786148              230.930195   
7 2024-11-09 00:00:00+00:00  230.658997              225.245599   
8 2024-11-10 00:00:00+00:00  230.539169              222.284007   
9 2024-11-11 00:00:00+00:00  230.424500              231.252375   

   Hybrid Model Pred  
0         231.495794  
1         229.897124  
2         228.827279  
3         231.064186  
4         229.399271  
5         228.394983  
6         230.829367  
7         229.034968  
8         228.062613  
9         230.672862  
